In [6]:
from amplify import gen_symbols, BinaryPoly, IsingPoly
from amplify import sum_poly
from amplify.constraint import equal_to
import matplotlib.pyplot as plt
from amplify import Solver
from amplify.client import FixstarsClient
import matplotlib.ticker as ticker   
import os

In [7]:
def q_annealing(n, c):
    s = gen_symbols(IsingPoly, n, n)

    # 確定しているもののスピンを確定------------------------------------------
    for i in range(n):
        for j in range(n):
            if c[i][j] == "B":
                s[i][j] = 1
            if c[i][j] == "W":
                s[i][j] = -1

    # QUBO1 コスト項　反強磁性イジングモデル-----------------------------------
    f = 0
    for i in range(n):
        for j in range(n):
            if i != n-1:
                f += s[i][j] * s[i+1][j]
            if j != n-1:
                f += s[i][j] * s[i][j+1]

    model = f  
    
    # 計算実行------------------------------------------------------------------------------------------
    client = FixstarsClient()
    with open("tokenfile.txt", "r") as f:
        token = f.readline()
    client.token = token
    client.parameters.timeout = 9000  # タイムアウト9秒
    solver = Solver(client)

    result = solver.solve(model)
    if len(result) == 0:
        raise RuntimeError("Any one of constraints is not satisfied.")

    energy, values = result[0].energy, result[0].values

    return int((2*n*(n-1) -energy)/2)

In [ ]:
"""
# atcoder入力形式
n = int(input())
c = [input() for i in range(n)]

"""

# テストケース読み込み
# https://www.dropbox.com/sh/nx3tnilzqz7df8a/AACpUzqOotp665-GQMdAbHJua/ABC193/F?dl=0&subfolder_nav_tracking=1

files = os.listdir("in")
for file in files:
    with open("in/" + file, "r") as f:
        n = int(f.readline())
        c = [f.readline() for i in range(n)]
    with open("out/" + file, "r") as f:
        ans = int(f.readline())

    try:
        result = q_annealing(n, c)
        print(file, "Amplify結果=", result, "正解=", ans)
    except:
        pass


02_sample.txt Amplify結果= 4 正解= 4
03_sample.txt Amplify結果= 40 正解= 40
06_small.txt Amplify結果= 81 正解= 81
08_small.txt Amplify結果= 39 正解= 39
09_small.txt Amplify結果= 84 正解= 84
10_small.txt Amplify結果= 39 正解= 39
11_small.txt Amplify結果= 62 正解= 62
12_small.txt Amplify結果= 7 正解= 7
13_small.txt Amplify結果= 73 正解= 73
14_small.txt Amplify結果= 10 正解= 10
16_small.txt Amplify結果= 48 正解= 48
18_large.txt Amplify結果= 5535 正解= 5535
19_large.txt Amplify結果= 3690 正解= 3690
20_large.txt Amplify結果= 103 正解= 103
22_large.txt Amplify結果= 4442 正解= 4442
23_large.txt Amplify結果= 8550 正解= 8550
24_large.txt Amplify結果= 14696 正解= 14702
25_large.txt Amplify結果= 8635 正解= 8635
26_large.txt Amplify結果= 13750 正解= 13750
28_max.txt Amplify結果= 11358 正解= 11358
29_max.txt Amplify結果= 12739 正解= 12739
30_max.txt Amplify結果= 13885 正解= 13885
